<a href="https://colab.research.google.com/github/tofaratifolayan/potato_resnet/blob/main/potato_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L -o potato-viral-disease-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/nirmalsankalana/potato-viral-disease-dataset


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  799M  100  799M    0     0  20.5M      0  0:00:38  0:00:38 --:--:-- 22.9M


In [ ]:
!unzip -q potato-viral-disease-dataset.zip -d potato-viral-disease-dataset

In [ ]:
!curl -L -o potato-healthy-dataset.zip\
https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/5m38z6jthb-1.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  238M  100  238M    0     0  14.3M      0  0:00:16  0:00:16 --:--:-- 16.7M


In [ ]:
!unzip -q potato-healthy-dataset.zip -d potato-viral-disease-dataset

In [ ]:
import os
import pandas as pd

file_paths = []
labels = []
class_dict = {"Potato___healthy": 0, "Potato___leafroll_virus": 1, "Potato___mosaic_virus": 2, "Potato___spindle_tuber_viroid": 3, "Potato__healthy_bulb": 4}
class_folders = os.listdir("/content/potato-viral-disease-dataset")
# Iterate over each class folder
for label, class_name in enumerate(class_folders):
    class_folder = os.path.join("/content/potato-viral-disease-dataset", class_name)

    # Check if it's a directory
    if os.path.isdir(class_folder):
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if not os.path.isdir(file_path):
              file_paths.append(file_path)
              # Store class name as label
              labels.append(class_dict[class_name])

# Create a DataFrame
df = pd.DataFrame({"file_path": file_paths, "label": labels})

# Display the first few rows
# pd.set_option('display.max_colwidth', None)
print(df)

                                              file_path  label
0     /content/potato-viral-disease-dataset/Potato__...      3
1     /content/potato-viral-disease-dataset/Potato__...      3
2     /content/potato-viral-disease-dataset/Potato__...      3
3     /content/potato-viral-disease-dataset/Potato__...      3
4     /content/potato-viral-disease-dataset/Potato__...      3
...                                                 ...    ...
1558  /content/potato-viral-disease-dataset/Potato__...      2
1559  /content/potato-viral-disease-dataset/Potato__...      2
1560  /content/potato-viral-disease-dataset/Potato__...      2
1561  /content/potato-viral-disease-dataset/Potato__...      2
1562  /content/potato-viral-disease-dataset/Potato__...      2

[1563 rows x 2 columns]


In [ ]:
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose([
    # Resize smaller edge to 256
    transforms.Resize(256),
    # Crop the center to 224x224
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # Normalize
    transforms.Normalize(mean=[0.5], std=[0.5])
])


class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = hw dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]

        # Open image and convert to RGB
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)  # Ensure it's a tensor

        return image, label

In [ ]:
import torch
import torch.nn as nn

class Residual(nn.Module):
    def __init__(self, numChannels, out_channels, third_conv=False, downsample=False):
        super().__init__()

        # conv1
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # conv2
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # relu
        self.relu = nn.ReLU()

        # 3rd conv
        self.third_conv = third_conv
        if third_conv:
            self.conv3 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
            self.bn3 = nn.BatchNorm2d(out_channels)


        self.downsample = nn.Conv2d(numChannels, out_channels, kernel_size=1, stride=1, bias=False) if numChannels != out_channels else None

    def forward(self, X):
        identity = X

        Y = self.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))

        if self.third_conv:
            Y = self.bn3(self.conv3(self.relu(Y)))

        if self.downsample:
            identity = self.downsample(X)

        Y += identity
        return self.relu(Y)

class PotatoNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual blocks
        self.residual1 = Residual(64, 64)
        self.residual2 = Residual(64, 128, downsample=False)
        self.residual3 = Residual(128, 256, downsample=False)
        self.residual4 = Residual(256, 512, downsample=False)

        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers
        self.fc1 = nn.Linear(512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.last_fc = nn.Linear(4096, num_classes)

    def forward(self, X):
        X = self.relu(self.bn1(self.conv1(X)))
        X = self.pool(X)

        X = self.residual1(X)
        X = self.residual2(X)
        X = self.residual3(X)
        X = self.residual4(X)

        X = self.global_pool(X)
        X = torch.flatten(X, 1)

        X = self.fc1(X)
        X = self.relu(X)
        X = self.fc2(X)
        X = self.relu(X)
        X = self.last_fc(X)

        return X


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_dataset = ImageDataset(train_df, transform=transform)
test_dataset = ImageDataset(test_df, transform=transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Training function
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

        print(f"Epoch {epoch+1}: Loss = {epoch_loss/len(train_loader):.4f}, Accuracy = {100 * correct / total:.2f}%")

# Evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


In [ ]:
import torch
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model = PotatoNet(num_classes=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Attach scheduler to optimizer
# Define loss function
criterion = nn.CrossEntropyLoss()

# def train
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()

    best_acc = 0  # Store best accuracy
    best_model_path = "best_model.pth"

    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Loss = {epoch_loss/len(train_loader):.4f}, Accuracy = {acc:.2f}%")

        # Adjust learning rate based on validation accuracy

        # **Save Best Model**
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), best_model_path)
            print(f"Best Model Saved at {best_model_path} with Accuracy: {best_acc:.2f}%")

    print(f"Training Complete! Best Accuracy: {best_acc:.2f}%")

# Train the model
train(model, train_dataloader, criterion, optimizer, device, num_epochs=10)

# Evaluate on test set
evaluate(model, test_dataloader, criterion, device)


Epoch 1/10: 100%|██████████| 20/20 [00:43<00:00,  2.16s/it, acc=63.2, loss=0.457]


Epoch 1: Loss = 0.9682, Accuracy = 63.20%
Best Model Saved at best_model.pth with Accuracy: 63.20%


Epoch 2/10: 100%|██████████| 20/20 [00:42<00:00,  2.14s/it, acc=85.4, loss=0.28]


Epoch 2: Loss = 0.3798, Accuracy = 85.36%
Best Model Saved at best_model.pth with Accuracy: 85.36%


Epoch 3/10: 100%|██████████| 20/20 [00:42<00:00,  2.15s/it, acc=84.4, loss=0.218]


...

Epoch 9/10: 100%|██████████| 20/20 [00:42<00:00,  2.12s/it, acc=90.3, loss=0.334]


Epoch 9: Loss = 0.2515, Accuracy = 90.32%


Epoch 10/10: 100%|██████████| 20/20 [00:43<00:00,  2.18s/it, acc=87.8, loss=0.414]

Epoch 10: Loss = 0.2742, Accuracy = 87.84%
Training Complete! Best Accuracy: 90.72%


Test Loss: 0.2279, Test Accuracy: 90.73%
